In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.datasets
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

import random

import os

In [17]:
class config:
    seed = 42
    device = "cuda:0"    
        
    lr = 1e-3
    epochs = 25
    batch_size = 32
    num_workers = 4
    train_5_folds = True

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything(config.seed)

In [18]:
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

device

device(type='cuda')

In [19]:
transform_train = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])

transform_validation = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])

transform  = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])

In [20]:
trainset = torchvision.datasets.CIFAR10(root='/data',
                                        train=True,
                                        download=True,
                                        transform=transform_train)

vaildset = torchvision.datasets.CIFAR10(root='/data',
                                        train=False,
                                        download=True,
                                        transform=transform_validation)
testset = torchvision.datasets.ImageFolder(root = "/content/drive/MyDrive/share/images/Statistical_Deep_Image",
                                           transform = transform)

Files already downloaded and verified
Files already downloaded and verified


In [21]:
train_loader = DataLoader(trainset,
                          batch_size = 32,
                          shuffle=True,
                          num_workers=2)

vaild_loader = DataLoader(vaildset,
                          batch_size = 64,
                          shuffle=True,
                          num_workers=2)


test_loader = DataLoader(testset,
                          batch_size=32,
                          shuffle=False,
                          num_workers=2)

In [22]:
classes = ('plane','automobile','bird','cat','deer',
           'dog','frog','horse','ship','truck')

모델 넣기 

In [23]:
class BasicBlock(nn.Module):
   def __init__(self, in_channels, out_channels, kernel_size=3):
       super(BasicBlock, self).__init__()


       # ❶ 합성곱층 정의
       self.c1 = nn.Conv2d(in_channels, out_channels, 
                           kernel_size=kernel_size, padding=1)
       self.c2 = nn.Conv2d(out_channels, out_channels, 
                           kernel_size=kernel_size, padding=1)

       self.downsample = nn.Conv2d(in_channels, out_channels, 
                                   kernel_size=1)
       
       # ❷ 배치 정규화층 정의
       self.bn1 = nn.BatchNorm2d(num_features=out_channels)
       self.bn2 = nn.BatchNorm2d(num_features=out_channels)

       self.relu = nn.ReLU()
   def forward(self, x):
       # ❸스킵 커넥션을 위해 초기 입력을 저장
       x_ = x

       x = self.c1(x)
       x = self.bn1(x)
       x = self.relu(x)
       x = self.c2(x)
       x = self.bn2(x)

       # ➍합성곱의 결과와 입력의 채널 수를 맞춤
       x_ = self.downsample(x_)

       # ➎합성곱층의 결과와 저장해놨던 입력값을 더해줌
       x += x_
       x = self.relu(x)

       return x


class ResNet(nn.Module):
   def __init__(self, num_classes=10):
       super(ResNet, self).__init__()


       # ❶ 기본 블록
       self.b1 = BasicBlock(in_channels=3, out_channels=64)
       self.b2 = BasicBlock(in_channels=64, out_channels=128)
       self.b3 = BasicBlock(in_channels=128, out_channels=256)


       # ❷ 풀링을 최댓값이 아닌 평균값으로
       self.pool = nn.AvgPool2d(kernel_size=2, stride=2) 

       # ❸ 분류기
       self.fc1 = nn.Linear(in_features=4096, out_features=2048)
       self.fc2 = nn.Linear(in_features=2048, out_features=512)
       self.fc3 = nn.Linear(in_features=512, out_features=num_classes)

       self.relu = nn.ReLU()
   def forward(self, x):
       # ❶ 기본 블록과 풀링층을 통과
       x = self.b1(x)
       x = self.pool(x)
       x = self.b2(x)
       x = self.pool(x)
       x = self.b3(x)
       x = self.pool(x)


       # ❷ 분류기의 입력으로 사용하기 위해 flatten
       x = torch.flatten(x, start_dim=1)

       # ❸ 분류기로 예측값 출력
       x = self.fc1(x)
       x = self.relu(x)
       x = self.fc2(x)
       x = self.relu(x)
       x = self.fc3(x)

       return x



In [24]:

net = ResNet(num_classes=10).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

total=0 
correct = 0

In [25]:
for epoch in range(50):
  running_loss = 0.0

  for i, data in enumerate(train_loader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()
    outputs= net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 750 ==749:
      print("Epoch: {},Batch : {}, Loss:{}".format(epoch+1, i+1, running_loss/2000))
      running_loss = 0.0

  with torch.no_grad():
    val_loss = 0.0
    for k, data1 in enumerate(vaild_loader, 0):
      val_inputs, val_label = data1[0].to(device), data1[1].to(device)
      val_output = net(val_inputs)
      v_loss = criterion(val_output, val_label)
      val_loss += v_loss
  print("validation loss {}".format(val_loss))

  with torch.no_grad():
    for data in test_loader:
      images, labels = data[0].to(device), data[1].to(device)
      outputs= net(images)
      _, predicted = torch.max(outputs.data,1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  print(100* correct/total)
  total=0
  correct=0


Epoch: 1,Batch : 750, Loss:0.609159421980381
Epoch: 1,Batch : 1500, Loss:0.4280039099752903
validation loss 153.9091033935547
24.35
Epoch: 2,Batch : 750, Loss:0.3406668953448534
Epoch: 2,Batch : 1500, Loss:0.3030681117922068
validation loss 116.82099151611328
29.7
Epoch: 3,Batch : 750, Loss:0.26363074673712256
Epoch: 3,Batch : 1500, Loss:0.2471133171617985
validation loss 106.47334289550781
31.45
Epoch: 4,Batch : 750, Loss:0.21894020695984365
Epoch: 4,Batch : 1500, Loss:0.21415723170340062
validation loss 96.43048858642578
35.75
Epoch: 5,Batch : 750, Loss:0.1870575524494052
Epoch: 5,Batch : 1500, Loss:0.18651565035432577
validation loss 86.54078674316406
35.9
Epoch: 6,Batch : 750, Loss:0.1660231494344771
Epoch: 6,Batch : 1500, Loss:0.16449183114618063
validation loss 78.4665756225586
37.65
Epoch: 7,Batch : 750, Loss:0.14786352048069237
Epoch: 7,Batch : 1500, Loss:0.1444020591787994
validation loss 77.96278381347656
38.95
Epoch: 8,Batch : 750, Loss:0.1273982113339007
Epoch: 8,Batch : 15

pretrained 넣기


In [26]:
PATH = '/content/drive/MyDrive/share/trained_model/cifar_resnet.pth'

In [27]:
#net = ResNet(num_classes=10).to(device)
#net.load_state_dict(torch.load(PATH))

In [28]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    c= (predicted == labels).squeeze()
    for i in range(4):
      label= labels[i]
      class_correct[label]+= c[i].item()
      class_total[label]+= 1

for i in range(10):
  print("Accuracy of {} : {} %".format(classes[i],100* class_correct[i]/class_total[i]))

Accuracy of plane : 82.14285714285714 %
Accuracy of automobile : 33.333333333333336 %
Accuracy of bird : 29.166666666666668 %
Accuracy of cat : 37.5 %
Accuracy of deer : 71.42857142857143 %
Accuracy of dog : 25.0 %
Accuracy of frog : 20.833333333333332 %
Accuracy of horse : 33.333333333333336 %
Accuracy of ship : 35.714285714285715 %
Accuracy of truck : 41.666666666666664 %


In [29]:
correct = 0
total = 0
with torch.no_grad():
  for data in train_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print(100* correct/total)

99.93


In [30]:
correct = 0
total = 0
with torch.no_grad():
  for data in vaild_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print(100* correct/total)

88.3


In [31]:
correct = 0
total = 0
with torch.no_grad():
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print(100* correct/total)

45.95
